 ## Prise en main de la librairie GEDCOM

### Conventions : 
Tag for Spouse FamiliyElement : 'FAMS'
$\\$
Tag for Childrens FamiliyElement : 'FAMC'

In [90]:
from gedcom.element.individual import IndividualElement
from gedcom.element.element import Element
from gedcom.parser import Parser
from gedcom.element.family import FamilyElement
import gedcom.tags
import numpy as np
import pandas as pd
import string
#from gedcom.element.element import Element
#from gedcom.element.file import FileElement
#from gedcom.element.object import ObjectElement

In [30]:
file_path = 'Queen_Eliz_II.ged'
gedcom_parser = Parser()
gedcom_parser.parse_file(file_path)
root_child_elements = gedcom_parser.get_root_child_elements()

def find_child_data(first_names,last_name):
    for element in root_child_elements:
        if isinstance(element, IndividualElement):
            if element.surname_match(last_name):
                if element.given_name_match(first_names):
                    L1 = []
                    L2 = []
                    (first, last) = element.get_name()
                    print("Information about " + first + " " + last + ":")
                    data = element.get_child_elements()
                    for child in data :
                        L1 += [child]
                        L2 += [child.get_tag()]
                    return L1, L2
    return "No matching person"

In [10]:
find_child_data("Elizabeth II Alexandra Mary", "Windsor")

Information about Elizabeth II Alexandra Mary Windsor:


([<gedcom.element.element.Element at 0x10866aac0>,
 ['RIN', '_UID', '_UPD', 'NAME', 'SEX', 'BIRT', 'FAMS', 'FAMC'])

In [11]:
def find_IndividualElement(first_names,last_name):
    """
    Get Individual element of ged with first names and last name

    Parameters
    ---
    first_names : str
        complete first names of the individual
    last_name : str
        last names of the individual 

    Returns 
    ---
    bool 
        Tells if the individual was found
    IndividualElement
        IndividualElement corresponding to the individual in the ged

    """
    for element in root_child_elements:
        if isinstance(element, IndividualElement):
            if element.surname_match(last_name):
                return (True, element)
    return [False, element]

In [12]:
def get_Spouse_FamilyElement(Individual):
    """
    Get Spouse FamilyElements of an individual

    Parameters
    ---
    Individual : IndividualElement

    Returns
    ---
    list of FamilyElement 
    """
    return gedcom_parser.get_families(Individual,"FAMS")
    

In [13]:
def get_Children_FamilyElement(Individual):
    """
    Get Childrens FamilyElements of an individual

    Parameters
    ---
    Individual : IndividualElement

    Returns
    ---
    list of FamilyElement 
    """
    return gedcom_parser.get_families(Individual,"FAMC")

In [14]:
def get_spouse(Individual):
    """
    Doesn't work
    """
    Family = get_Spouse_FamilyElement(Individual)[0]
    return gedcom_parser.get_family_members(Family,"FAMILY_MEMBERS_TYPE_ALL")

### Tests

In [15]:
get_Spouse_FamilyElement(find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1])

In [16]:
get_Spouse_FamilyElement(find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1])[0].get_tag()

'FAM'

In [17]:
get_Children_FamilyElement(find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1])

In [18]:
A = find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1]
B = find_IndividualElement("George VI", "Windsor")[1]
gedcom_parser.find_path_to_ancestor(A,B,None)[0].get_name()

('George V', 'Windsor')

In [19]:
C = find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1]
D = find_IndividualElement("George V", "Windsor")[1]
gedcom_parser.find_path_to_ancestor(A,B,None)[0].get_name()

('George V', 'Windsor')

In [20]:
E = find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1]
F = find_IndividualElement("Edward VII", "Wettin")[1]
gedcom_parser.find_path_to_ancestor(A,B,None)[0].get_name()

('George V', 'Windsor')

In [21]:
get_spouse(find_IndividualElement("George VI", "Windsor")[1])

In [22]:
get_spouse(find_IndividualElement("George VI", "Windsor")[1])[0].get_name()

('George V', 'Windsor')

In [23]:
get_spouse(find_IndividualElement("George VI", "Windsor")[1])[1].get_name()

('Mary of Teck (May)', '')

In [24]:
get_spouse(find_IndividualElement("George VI", "Windsor")[1])[2].get_name()

('George VI', 'Windsor')

In [25]:
get_spouse(find_IndividualElement('Mary of Teck (May)', '')[1])[0].get_name()

('SIR WILLIAM VII KNIGHT ENGLAND', 'Gascoigne')

### Fonctions Parents/Childrens

In [26]:
def get_all_Spouse_FamilyElement():
    Spouse_FamilyElements = []
    N = 0
    for element in root_child_elements:
        if isinstance(element, FamilyElement):
            if element.get_tag() == gedcom.tags.GEDCOM_TAG_FAMILY_SPOUSE :
                Spouse_FamilyElements += [element]
    return Spouse_FamilyElements


In [27]:
get_all_Spouse_FamilyElement()

[]

In [28]:
get_Childrens(find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1])

NameError: name 'get_Childrens' is not defined

In [ ]:
gedcom_parser.get_parents(find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1])

### Raisonnement : 
Construction d'une Dataframe panda Individual / Family1 / Family2 $\\$
Construction d'une Dataframe panda Family / Children1 / FamilyChildren2 / ... $\\$
Construction d'une Dataframe panda Family / Mother / Father $\\$

Parcours du graphe : 
On part d'un des 2, on part des 2 côté avec les 2 dernières dataframe jusqu'à trouver la famille du 2e

In [93]:
def get_IndivFamily_table():
    T = []
    for element in root_child_elements:
        if isinstance(element, IndividualElement):
            L = [element.get_pointer()]
            for child_element in element.get_child_elements() :
                if child_element.get_tag() == gedcom.tags.GEDCOM_TAG_FAMILY_SPOUSE :
                    L += [child_element.get_value()]
                elif child_element.get_tag() == gedcom.tags.GEDCOM_TAG_FAMILY_CHILD :
                    L += [child_element.get_value()]
            T += [L]
    return T



In [98]:
table = get_IndivFamily_table()
full_table = [line+['None']*(3-len(line)) for line in table]
full_table

[['@I101@', '@F285@', '@F286@'],
 ['@I103@', '@F286@', '@F287@'],
 ['@I155@', '@F78@', 'None'],
 ['@I168@', '@F75@', '@F76@'],
 ['@I169@', '@F75@', '@F209@'],
 ['@I170@', '@F76@', '@F77@'],
 ['@I171@', '@F76@', '@F214@'],
 ['@I172@', '@F77@', '@F81@'],
 ['@I173@', '@F77@', '@F156@'],
 ['@I174@', '@F313@', '@F2419@'],
 ['@I175@', '@F314@', '@F315@'],
 ['@I178@', '@F315@', 'None'],
 ['@I179@', '@F81@', '@F82@'],
 ['@I180@', '@F81@', '@F3174@'],
 ['@I181@', '@F82@', '@F83@'],
 ['@I182@', '@F82@', 'None'],
 ['@I183@', '@F83@', '@F84@'],
 ['@I184@', '@F84@', '@F85@'],
 ['@I185@', '@F84@', '@F3501@'],
 ['@I186@', '@F85@', '@F89@'],
 ['@I193@', '@F89@', '@F90@'],
 ['@I194@', '@F89@', 'None'],
 ['@I195@', '@F83@', 'None'],
 ['@I196@', '@F85@', 'None'],
 ['@I197@', '@F90@', '@F91@'],
 ['@I198@', '@F90@', 'None'],
 ['@I199@', '@F91@', '@F92@'],
 ['@I200@', '@F91@', 'None'],
 ['@I201@', '@F92@', '@F93@'],
 ['@I202@', '@F92@', '@F3095@'],
 ['@I203@', '@F93@', '@F94@'],
 ['@I204@', '@F93@', 'None']

In [101]:
INDI_elements = [full_table[k][0] for k in range(len(full_table))]
FAMS_elements = [full_table[k][1] for k in range(len(full_table))]
FAMC_elements = [full_table[k][2] for k in range(len(full_table))]

In [103]:
df = pd.DataFrame(
    {
        'INDI' : [],
        'FAMS' : [],
        'FAMC' : [],
    })
df

,INDI,FAMS,FAMC


In [112]:
df['INDI'] = INDI_elements
df['FAMS'] = FAMS_elements
df['FAMC'] = FAMC_elements
df = df.set_index('INDI')
df

,FAMS,FAMC
INDI,,
@I101@,@F285@,@F286@
@I103@,@F286@,@F287@
@I155@,@F78@,None
@I168@,@F75@,@F76@
@I169@,@F75@,@F209@
...,...,...
@I11259@,@F6018@,@F6019@
@I11260@,@F6019@,None
@I11261@,@F6019@,None
